# GEOG 308 - PySAL Demo for Spatial Data Preparation & GeoVisualization

<font size="3">
Dr. Aynaz Lotfata <br>
Visiting Assistant Professor <br>
Miami University, OH 
</font>

This short demo will show you how to:
1. Create geographic data from flat text or raw geometries
2. Write out that data to file
3. Plot the data on a map
4. Geovisualization by a specific attribute

Following Python packages will be used for the tutorial

In [ ]:
import geopandas as gpd # to read/write spatial data
import matplotlib.pyplot as plt # to visualize data
import pandas # to read/write plain tables

# to display a few webpages within the notebook
from IPython.display import IFrame
from shapely import wkb 

import libpysal as lp #pysal library
import seaborn as sbn #statistical distribution plotting

%matplotlib inline

# Reading data

Following Points to note:
1. In many cases, you may have *prepared* data in format such as shapefiles, `tiff` rasters, geopackage `.gpkg` files.
2. A common issue when working with databases having *raw* data is how to create geodata from flat text files.

We will use sample data from [insideairbnb.com](https://insideairbnb.com) which contains features & prices for AirBnB locations in *Austin, TX*

In [ ]:
listings = pandas.read_csv('../data/listings.csv.gz')
neighborhoods = pandas.read_csv('../data/neighborhoods.csv')

Let us see how many `listings` and how many `neighborhoods` do the two datasets contain:

In [ ]:
print(listings.shape) 
print(neighborhoods.shape)

For the first dataset, the `listings` data, records are provided with information about the latitude & longitude of the listing.

Let us see the first 5 rows of `listings` data:

In [ ]:
listings.head()

Let us see the what columns are in the `listings` data

In [ ]:
list(listings.columns)

From all the columns, following columns that contain geographic information can be used from the above dataframe:
1. city
2. state
3. zipcode
4. hood, which means neighborhood
5. country
6. latitude & longitude, which are the coordinates of the listing
geometry, which is the well-known text representation of the point iself.

Now, let us look at the `neighborhoods` data
1. Examine the first 5 rows in this dataset
2. Identify names of columns in this dataset

In [ ]:
neighborhoods.head()

In [ ]:
list(neighborhoods.columns)

What is the **wkb** field above?
1. It is the only field that is useful in constructing the geometric representation of the data.
2. It is called *well-known binary representation* - a common format to represent geographic information.
3. `neighborhoods` are represented by *polygons*. 
4. **wkb** field above encodes the shape of these *polygons*

# Creating Geometries from raw co-ordinates

For the `listings` dataset, `geopandas` is used to construct a *"geodataframe"*, useful for working with geographic data, directly from coordinates. 

We can do this using the `points_from_xy` function using *latitudes* and *longitudes* data from `listings`:

In [ ]:
geometries = gpd.points_from_xy(listings.longitude, listings.latitude)

Then we make a geodataframe for `listings` as follows:

In [ ]:
listings = gpd.GeoDataFrame(listings, geometry=geometries)

How about for the `neighborhoods` data:
1. For the `neighborhoods` data, we must parse the well-known binary. 
2. The package that handles geometric data in Python is called `shapely`. 3. Using the `wkb` module in `shapely`, we can parse the well-known binary and obtain a geometric representation for the neighborhoods:

In [ ]:
neighborhoods['geometry'] = neighborhoods.wkb.apply(lambda shape: wkb.loads(shape, hex=True))

In [ ]:
neighborhoods.geometry[0]

In [ ]:
neighborhoods.geometry[1]

In [ ]:
neighborhoods = gpd.GeoDataFrame(neighborhoods)
neighborhoods.drop('wkb', axis=1, inplace=True)

What kind of geometries do each type of data represent?
1. `listings` is a **POINT**
2. `neighborhoods` is a **MULTIPOLYGON**

In [ ]:
listings.geometry[[0]]

In [ ]:
neighborhoods.geometry[[1]]

Let us pick an appropriate co-ordinate reference system for data in raw latitude/longitude values. This can be `espg:4326` format -- details available at [epsg.io/4326](https://epsg.io/4326) (WGS 84 -- WGS84 - World Geodetic System 1984, used in GPS)

In [ ]:
listings.crs = {'init':'epsg:4326'}
neighborhoods.crs = {'init': 'epsg:4326'}

Then, we can write them out to file using the `to_file` method on a geodataframe: 

In [ ]:
neighborhoods.to_file('../data/neighborhoods.gpkg', driver='GPKG')
listings.to_file('../data/listings.gpkg', driver='GPKG')

# Plotting neighborhoods and listings together

Let us pick an appropriate co-ordinate reference system for data in raw latitude/longitude values. This can be `espg:3857` format -- details available at [epsg.io/3857](https://epsg.io/3857) (WGS 84 / Pseudo-Mercator -- Spherical Mercator, Google Maps, OpenStreetMap, Bing, ArcGIS, ESRI)

In [ ]:
listings = listings.to_crs(epsg=3857)
neighborhoods = neighborhoods.to_crs(epsg=3857)

In [ ]:
plt.figure(figsize=(10, 10))
neighborhoods.boundary.plot(ax=plt.gca(), color='orangered')
listings.plot(ax=plt.gca(), marker='.', markersize=5, color='green')

# Geo-visualization 

Read files saved previously and drop $ sign in price of listing

In [ ]:
df = gpd.read_file('../data/neighborhoods.gpkg')
listings = gpd.read_file('../data/listings.gpkg')
listings['price'] = listings.price.str.replace('$', '').str.replace(',','_').astype(float)

Since price is available by `listings` we will have to calculate median price for every neighborhood before geovisualization

In [ ]:
median_price = gpd.sjoin(listings[['price', 'geometry']], df, op='within')\
                  .groupby('index_right').price.median()
df['median_pri'] = median_price.values

In [ ]:
df['median_pri'].fillna((df['median_pri'].mean()), inplace=True)

Plotting the distribution of median price of all neighborhoods

In [ ]:
sbn.distplot(df['median_pri'])

Geovisualizing neighborhoods by median prices

In [ ]:
fig, ax = plt.subplots(figsize=(12,10), subplot_kw={'aspect':'equal'})
df.plot(column='median_pri', scheme='Quantiles',  legend=True, ax=ax)